In [209]:
import pandas as pd
import numpy as np
from utils import pickle_obj

In [5]:
!ls "./assets/divaScrape"

SU  onlyAuthors.csv  sodertorn	uppsala


In [229]:
def all_auth_str(author, coauthor):
    all_auth_list = list()
    for a, ca in zip(author, coauthor):
        if ca is not np.nan:
            all_auth = str(a) +": " + str(ca)
            all_auth_list.append(all_auth)
        else:
            all_auth_list.append(a)
    return all_auth_list
    

In [306]:
def reformat_str_convnetion(authors):
    all_auth_list = list()
    for auth in authors:
        split_a = auth.split(": ")
        a_str = str(split_a[0])
        for a in split_a[1:]:
            a_str += ":" + str(a)
        all_auth_list.append(a_str)
    return all_auth_list

In [307]:
def all_unique_names(all_auth_list):
    unique_auth = []
    for author in all_auth_list:
        auth_split = author.split(":") 
        for a in auth_split:
            unique_auth.append(a)
        
    return list(set(unique_auth))
    

In [308]:
def get_id_to_auth_dict(unique_auth, prefix):
    id_to_auth = dict()
    for i, ua in enumerate(unique_auth):
        id_to_auth["u" + str(prefix) + str(i)] = ua
    return id_to_auth


In [309]:
def get_auth_to_id_dict(id_to_auth):
    return {v:k for v,k in zip(list(id_to_auth.values()), list(id_to_auth.keys()))}

In [310]:
def get_id_str(all_auth_list, auth_to_id):
    auth_id = list()
    for author in all_auth_list:
        auth_split = author.split(":")
        id_str = auth_to_id[auth_split[0]]
        for AuthS in auth_split[1:]:
            id_str += ":" + auth_to_id[AuthS]
        auth_id.append(id_str)
    return auth_id
    
    

In [319]:
def get_doc_docid(length, suffix):
    doc_id = list()
    for i in range(length):
        doc_id.append(str(i) + "_" + str(suffix))
    return doc_id   
    

In [326]:
def get_DF(abstract, auth_id, doc_id, all_auth_list):
    DF = pd.DataFrame(data=list(zip(abstract, auth_id, doc_id, all_auth_list)), columns=["Abstracts", "Auth_id", "Doc_id", "Auth_name"])
    return DF[~DF.Abstracts.isnull()].reset_index().drop(['index'], axis=1)

In [327]:


def get_nlp_data(scrapeData, prefix):

    author = scrapeData.Author.values
    coauthor = scrapeData.CoAuthor.values

    all_auth_list = all_auth_str(author, coauthor)
    
    print(len(all_auth_list))
    all_auth_list = reformat_str_convnetion(all_auth_list)
    
    print(len(all_auth_list))
    
    #print("--------")

    unique_auth = all_unique_names(all_auth_list)
    

    id_to_auth = get_id_to_auth_dict(unique_auth, prefix)

    auth_to_id = get_auth_to_id_dict(id_to_auth)

    auth_id = get_id_str(all_auth_list, auth_to_id)

    doc_id = get_doc_docid(len(scrapeData.Title), prefix)

    abstract = list(scrapeData.Abstract.values)

    DF = get_DF(abstract, auth_id, doc_id, all_auth_list)
    
    return DF, id_to_auth, auth_to_id
    
    
    

In [328]:
SUscrape = pd.read_csv("./assets/divaScrape/SU")
Uppsalascrape = pd.read_csv("./assets/divaScrape/uppsala")
Sodertornscrape = pd.read_csv("./assets/divaScrape/sodertorn")  

In [329]:
DF_SU, id_to_auth_SU, auth_to_id_SU = get_nlp_data(SUscrape, "SU")
DF_uppsala, id_to_auth_uppsala, auth_to_id_uppsala = get_nlp_data(Uppsalascrape, "Uppsala")
DF_sodertorn, id_to_auth_sodertorn, auth_to_id_sodertorn = get_nlp_data(Sodertornscrape, "sodertorn")

71581
71581
125548
125548
9227
9227


In [330]:
DF_SU

,Abstracts,Auth_id,Doc_id,Auth_name
0,We calculate the Josephson current density j (...,uSU14622:uSU92268:uSU62150,0_SU,"A. Madsen, Kevin:J. Bergholtz, Emil:Brouwer, P..."
1,There is agreement in research about the need ...,uSU38475,1_SU,"A. Manneh, Ilana"
2,Anthropomorphisms are widespread at all levels...,uSU38475:uSU54522:uSU54709:uSU83623,2_SU,"A. Manneh, Ilana:Hamza, Karim:Rundgren, Carl-J..."
3,"In this study, we explore the issues and chall...",uSU38475:uSU54709:uSU85386:uSU83623,3_SU,"A. Manneh, Ilana:Rundgren, Carl-Johan:Hamza, K..."
4,Tony Atkinson is universally celebrated for hi...,uSU75853:uSU35256:uSU38889:uSU98071:uSU43610:u...,5_SU,"Aaberge, Rolf:Bourguignon, François:Brandolini..."
...,...,...,...,...
44102,This paper seeks to represent the tropical sho...,uSU9392:uSU33872:uSU97270,71574_SU,"Žagar, Nedjeljka:Andersson, Erik:Fisher, Michael"
44103,In this study we question the relative importa...,uSU9392:uSU15459:uSU86959,71575_SU,"Žagar, Nedjeljka:Gustafsson, Nils:Källén, Erland"
44104,Geostrophic adjustment theory predicts that wi...,uSU9392:uSU15459:uSU86959,71576_SU,"Žagar, Nedjeljka:Gustafsson, Nils:Källén, Erland"
44105,Musik med texter om samhället.,uSU79401,71577_SU,"Jensen, Tommy"


In [331]:
DF_uppsala.to_csv("./assets/dataframes/uppsalaDf")
DF_SU.to_csv("./assets/dataframes/suDf")
DF_sodertorn.to_csv("./assets/dataframes/sodertornDf")

In [332]:
pickle_o = pickle_obj()
pickle_o.save(id_to_auth_SU, "./assets/dictionaries/id_to_auth_SU")
pickle_o.save(id_to_auth_uppsala, "./assets/dictionaries/id_to_auth_uppsala")
pickle_o.save(id_to_auth_sodertorn, "./assets/dictionaries/id_to_auth_sodertorn")

In [ ]:
l

In [64]:
DF_SU = pd.DataFrame(data=list(zip(abstract, auth_id, doc_id)), columns=["Abstract", "Auth_id", "Doc_id"])

In [74]:
DF_SU = DF_SU[~DF_SU.Abstract.isnull()]

In [76]:
DF_SU = DF_SU.reset_index()

In [77]:
DF_SU

,index,Abstract,Auth_id,Doc_id
0,0,We calculate the Josephson current density j (...,SU14623: SU92266: SU62147,SUD0
1,1,There is agreement in research about the need ...,SU38475,SUD1
2,2,Anthropomorphisms are widespread at all levels...,SU38475: SU54521: SU54708: SU83620,SUD2
3,3,"In this study, we explore the issues and chall...",SU38475: SU54708: SU85384: SU83620,SUD3
4,5,Tony Atkinson is universally celebrated for hi...,SU75851: SU35257: SU38889: SU98070: SU43610: S...,SUD5
...,...,...,...,...
44102,71574,This paper seeks to represent the tropical sho...,SU9393: SU33874: SU97269,SUD71574
44103,71575,In this study we question the relative importa...,SU9393: SU15460: SU86957,SUD71575
44104,71576,Geostrophic adjustment theory predicts that wi...,SU9393: SU15460: SU86957,SUD71576
44105,71577,Musik med texter om samhället.,SU79398,SUD71577


In [41]:
id_to_auth_su

{'SU0': 'Bernlochner, Florian U.',
 'SU1': 'Moses, P. G.',
 'SU2': 'Elykov, A.',
 'SU3': 'Rorai, Cecilia',
 'SU4': 'Kolesnikov, A.',
 'SU5': 'Aimbetov, Izzet',
 'SU6': 'Sarropoulou, Elena',
 'SU7': 'Lindbohm, Joni V.',
 'SU8': 'Wanntorp, L.',
 'SU9': 'Habing, H. J.',
 'SU10': 'Davis, Thompson E.',
 'SU11': 'Lepom, Peter',
 'SU12': 'Mortin, Jonas',
 'SU13': 'Ornellas, Adriana',
 'SU14': 'Hinchcliffe, William',
 'SU15': 'Robador, Alberto',
 'SU16': 'Hedqvist, Anders',
 'SU17': 'Vikström, Anna C',
 'SU18': 'Komar, N.',
 'SU19': 'Sanchez-Fernandez, C.',
 'SU20': 'Cortes, G.',
 'SU21': 'Nagamine, K.',
 'SU22': 'Dominguez Espinosa, Alejandra de Carmen',
 'SU23': 'van der Meer, Christianne',
 'SU24': 'Kaaks, Rudolf',
 'SU25': 'Roumeliotaki, Theano',
 'SU26': 'Reddy, S.',
 'SU27': 'Jokela, M.',
 'SU28': ' Kononowicz,  A',
 'SU29': 'Lyu, Wenhui',
 'SU30': 'Koptekin, Dilek',
 'SU31': 'McGrath, Lisa',
 'SU32': 'Saroukos, Spyridon',
 'SU33': 'Holmström, Tim H.',
 'SU34': 'Schaanning, M. T.',
 'SU3

In [40]:
auth_to_id_su

{'Bernlochner, Florian U.': 'SU0',
 'Moses, P. G.': 'SU1',
 'Elykov, A.': 'SU2',
 'Rorai, Cecilia': 'SU3',
 'Kolesnikov, A.': 'SU4',
 'Aimbetov, Izzet': 'SU5',
 'Sarropoulou, Elena': 'SU6',
 'Lindbohm, Joni V.': 'SU7',
 'Wanntorp, L.': 'SU8',
 'Habing, H. J.': 'SU9',
 'Davis, Thompson E.': 'SU10',
 'Lepom, Peter': 'SU11',
 'Mortin, Jonas': 'SU12',
 'Ornellas, Adriana': 'SU13',
 'Hinchcliffe, William': 'SU14',
 'Robador, Alberto': 'SU15',
 'Hedqvist, Anders': 'SU16',
 'Vikström, Anna C': 'SU17',
 'Komar, N.': 'SU18',
 'Sanchez-Fernandez, C.': 'SU19',
 'Cortes, G.': 'SU20',
 'Nagamine, K.': 'SU21',
 'Dominguez Espinosa, Alejandra de Carmen': 'SU22',
 'van der Meer, Christianne': 'SU23',
 'Kaaks, Rudolf': 'SU24',
 'Roumeliotaki, Theano': 'SU25',
 'Reddy, S.': 'SU26',
 'Jokela, M.': 'SU27',
 ' Kononowicz,  A': 'SU28',
 'Lyu, Wenhui': 'SU29',
 'Koptekin, Dilek': 'SU30',
 'McGrath, Lisa': 'SU31',
 'Saroukos, Spyridon': 'SU32',
 'Holmström, Tim H.': 'SU33',
 'Schaanning, M. T.': 'SU34',
 'Glo